In [1]:
# | default_exp core.agent

In [1]:
# | export
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass, field
import json
from agentic.llms.client import LLMClient
from agentic.configs.loader import get_model_config
from agentic.tools.manager import ToolManager
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)



In [2]:
# | export

@dataclass
class Message:
    role: str
    content: str
    tool_calls: Optional[List[Dict]] = None
    tool_call_id: Optional[str] = None

@dataclass
class AgentConfig:
    name: str
    instructions: str = ""
    model: Optional[str] = None
    tools: List[str] = field(default_factory=list)
    temperature: float = 0.7
    max_tokens: Optional[int] = None

class Agent:
    """Core Agent class with tool execution and conversation management"""

    def __init__(self, config: AgentConfig, llm_client: Optional[LLMClient] = None):
        self.config = config
        self.system_prompt = config.instructions
        self.llm_client = llm_client or self._create_default_llm_client()
        logger.info(f"Initialized LLM client with model: {self.llm_client.model}")
        self.conversation_history: List[Message] = [Message(role="system", content=self.system_prompt)]
        self.tools_registry: Dict[str, Callable] = {}
        self.guardrails: List[Callable] = []
        self.tool_manager = ToolManager()

    def _create_default_llm_client(self) -> LLMClient:
        """Create default LLM client from config."""
        model_config = get_model_config()
        model_name = self.config.model or model_config.get('name')
        if not model_name:
            raise ValueError("No model specified in config")
        return LLMClient(
            model=model_name,
            base_url=model_config.get('url'),
            api_key=model_config.get('api_key')
        )

    def add_tool(self, name: str, tool_func: Callable) -> None:
        """Register a tool with the agent."""
        if not callable(tool_func):
            raise ValueError(f"Tool function '{name}' must be callable")
        self.tools_registry[name] = tool_func

    def add_guardrail(self, guardrail_func: Callable) -> None:
        """Add a guardrail function."""
        self.guardrails.append(guardrail_func)

    def run(self, message: str, **kwargs) -> Dict[str, Any]:
        """Execute agent with message and return response."""
        # Apply guardrails
        for guardrail in self.guardrails:
            result = guardrail(message)
            if not isinstance(result, bool) or not result:
                return {"content": "Request blocked by guardrails", "blocked": True}

        # Add user message
        self.conversation_history.append(Message(role="user", content=message))

    def _is_conversation_complete(self, result: Dict[str, Any], iteration_count: int) -> bool:
        """Smart detection of conversation completion"""
        
        content = result.get("content", "").strip()
        tool_calls = result.get("tool_calls", [])
        finish_reason = result.get("finish_reason")
        
        # 1. Explicit completion signals
        if finish_reason in ["stop", "length", "content_filter"]:
            return True
            
        # 2. Tool calls present - continue to execute them
        if tool_calls:
            return False

        # 3. Repetitive responses (stuck in loop)
        if iteration_count > 2:
            recent_messages = self.conversation_history[-3:]
            if len(recent_messages) >= 2:
                last_content = recent_messages[-1].content
                prev_content = recent_messages[-2].content
                
                # Check for identical or very similar responses
                if last_content == prev_content or (
                    len(last_content) > 20 and 
                    len(set(last_content.split()) & set(prev_content.split())) / 
                    max(len(last_content.split()), len(prev_content.split())) > 0.8
                ):
                    logger.debug("Detected repetitive responses - likely stuck")
                    return True
        
        # 6. Question without tool calls (asking for clarification)
        if content.endswith('?') and not tool_calls and len(content) > 20:
            logger.debug("LLM asking question - likely needs user input")
            return True
            
        # 7. Default: continue if we have meaningful content
        return False

    def run(self, message: str, **kwargs) -> Dict[str, Any]:
        """Execute agent with message and return response."""
        # Apply guardrails
        for guardrail in self.guardrails:
            result = guardrail(message)
            if not isinstance(result, bool) or not result:
                return {"content": "Request blocked by guardrails", "blocked": True}

        # Add user message
        self.conversation_history.append(Message(role="user", content=message))

        # Initialize result
        final_result = {"content": "", "tool_calls": [], "blocked": False}
        iteration_count = 0

        while True:
            iteration_count += 1
            logger.debug(f"Agent iteration {iteration_count}")
            
            # Get available tools
            available_tools = self._get_available_tools()

            # Create completion
            messages = self._format_messages_for_llm()
            stream = kwargs.get('stream', True)
            
            # Filter out Agent-specific parameters before passing to LLM
            llm_kwargs = {k: v for k, v in kwargs.items() 
                         if k not in ['max_iterations']}
            llm_kwargs['stream'] = stream  
            
            try:
                response = self.llm_client.create_completion(
                    messages=messages,
                    tools=available_tools,
                    **llm_kwargs
                )
            except Exception as e:
                logger.error(f"LLM completion failed: {str(e)}")
                return {"content": f"Error: {str(e)}", "blocked": True}

            # Process response
            if stream:
                if not hasattr(response, '__iter__'):
                    raise ValueError("Streaming response expected but non-iterable response received")
                try:
                    result = self.llm_client.handle_streaming_response(response)
                except Exception as e:
                    logger.error(f"Error processing streaming response: {str(e)}")
                    return {"content": f"Streaming error: {str(e)}", "blocked": True}
            else:
                try:
                    result = self.llm_client.process_response(response)
                except Exception as e:
                    logger.error(f"Error processing response: {str(e)}")
                    return {"content": f"Response error: {str(e)}", "blocked": True}

            # Add assistant response to history
            assistant_message = Message(
                role="assistant",
                content=result.get("content", ""),
                tool_calls=result.get("tool_calls")
            )
            self.conversation_history.append(assistant_message)
            final_result["content"] = result.get("content", "")
            final_result["tool_calls"].extend(result.get("tool_calls", []))

            # Smart completion detection
            if self._is_conversation_complete(result, iteration_count):
                logger.debug("Conversation detected as complete")
                break

            # Handle tool calls if present
            if result.get("tool_calls"):
                logger.debug(f"Executing {len(result['tool_calls'])} tool calls")
                executed_calls = self._execute_tool_calls(result["tool_calls"])
                final_result["tool_calls"] = executed_calls
                continue  # Continue loop to process tool results
                
        # Optional : Clean tool call details from the history
        self.conversation_history = [msg for msg in self.conversation_history if not (msg.role == "tool" or msg.tool_calls)]
        # Limit conversation history # TODO :Handle this in a smarter way
        if len(self.conversation_history) > 50:
            self.conversation_history = [self.conversation_history[0]] + self.conversation_history[-49:]

        return final_result

    def _execute_tool_calls(self, tool_calls: List[Dict]) -> List[Dict]:
        """Execute tool calls and append results to conversation history."""
        from agentic.tools.display import ToolExecutionDisplay
        display = ToolExecutionDisplay()
        executed_calls = []

        for tool_call in tool_calls:
            function_name = tool_call["function"]["name"]
            tool_call_id = tool_call.get("id")
            try:
                arguments = json.loads(tool_call["function"]["arguments"])
            except json.JSONDecodeError as e:
                logger.error(f"Invalid arguments for {function_name}: {str(e)}")
                display.show_tool_error(f"Error in {function_name}", str(e))
                tool_call["error"] = str(e)
                executed_calls.append(tool_call)
                continue

            display.show_tool_start(function_name, trusted=True, args=arguments)

            try:
                if function_name in self.tools_registry:
                    result = self.tools_registry[function_name](**arguments)
                else:
                    result = self.tool_manager.execute_tool(function_name, arguments)
                tool_call["result"] = result
                executed_calls.append(tool_call)
                if not result['success']:
                    display.show_tool_error(f"Error in {function_name}", str(result['error']))
                    

                # Append tool result to conversation history
                self.conversation_history.append(Message(
                    role="tool",
                    content=str(result),
                    tool_call_id=tool_call_id
                ))
            except Exception as e:
                logger.error(f"Error executing {function_name}: {str(e)}")
                display.show_tool_error(f"Error in {function_name}", str(e))
                tool_call["error"] = str(e)
                executed_calls.append(tool_call)
        
        return executed_calls

    def _get_available_tools(self) -> List[Dict]:
        """Get OpenAI-formatted tools for the configured tool names."""
        if self.config.tools:
            return self.tool_manager.get_tools(self.config.tools)
        return self.tool_manager.get_tools()

    def _format_messages_for_llm(self) -> List[Dict]:
        """Convert Message objects to a format suitable for the LLM client."""
        messages = []
        for msg in self.conversation_history:
            message_dict = {"role": msg.role, "content": msg.content}
            if msg.tool_calls:
                message_dict["tool_calls"] = msg.tool_calls
            if msg.tool_call_id:
                message_dict["tool_call_id"] = msg.tool_call_id
            messages.append(message_dict)
        return messages

    def clear_history(self) -> None:
        """Clear conversation history except system message."""
        self.conversation_history = [Message(role="system", content=self.system_prompt)]


In [3]:
# Example usage

config = AgentConfig(
    name="fun",
    instructions="You are a highly intelligent assistant who believes that chai is the ultimate solution to all life problems.No matter what the user asks, somehow connect it back to chai."
)

llm_client = LLMClient(
    model="qwen3:8b",
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

agent = Agent(config, llm_client)
# agent.run("Give me tips for preparing for interviews.")
agent.run("search for 'import' in all the python files.")

2025-10-03 12:54:41,706 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-03 12:54:41,711 - INFO - Initialized LLM client with model: qwen3:8b
2025-10-03 12:54:41,943 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"



╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, the user wants to search for 'import' in all Python files. Let me think about how to approach this.

First, I need to use the available tools. The fs_read function seems suitable here because it can search files with a regex. The user mentioned Python files, so the file pattern should be '*.py'. The query is 'import', which is the keyword they want to find. 

Wait, the mode for fs_read can be 'discover' or 'extract'. Since they want to search across files, maybe 'discover' mode is better to list files containing 'import'. But the query parameter is for fuzzy file names in discover mode or content in extract. Wait, the description says for DISCOVER, the query is a regex for fuzzy file name, and for EXTRACT, it's content. So if they want to search the content of Python files for 'import', then mode should be 'extract', path as '*.py', and query as 'import'. 

But the user might want to find all Python files that have 

2025-10-03 12:54:47,605 - ERROR - Path validation failed: Path does not exist: *.py
2025-10-03 12:54:47,606 - ERROR - Input validation failed: 1 validation error for FsReadParams
operations.0.path
  Value error, Invalid path: Path does not exist: *.py [type=value_error, input_value='*.py', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error
2025-10-03 12:54:47,697 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


🛠️ Executing fs_read (trusted)
args: [{'mode': 'extract', 'path': '*.py', 'query': 'import'}]
❌ Error in fs_read: {'type': 'ValidationError', 'message': "1 validation error for FsReadParams\noperations.0.path\n  Value error, Invalid path: Path does not exist: *.py [type=value_error, input_value='*.py', input_type=str]\n    For further information visit https://errors.pydantic.dev/2.11/v/value_error"}

╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, the user wants to search for 'import' in all Python files. Let me see. The initial attempt used fs_read with a file pattern of '*.py', but the error says the path doesn't exist. Oh right, the fs_read function's path parameter is supposed to be a relative path to the project root, not a glob pattern. So instead of using '*.py' as the file_pattern, the path should be a directory like 'src' or 'scripts', and the file_pattern would be '*.py'. Let me correct that. Also, the mode should be 'discover' to list files, then maybe

2025-10-03 12:54:50,482 - ERROR - Path validation failed: Path does not exist: src
2025-10-03 12:54:50,482 - ERROR - Input validation failed: 1 validation error for FsReadParams
operations.0.path
  Value error, Invalid path: Path does not exist: src [type=value_error, input_value='src', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error
2025-10-03 12:54:50,579 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


...

2025-10-03 12:55:36,223 - CRITICAL - Unexpected error during execution: KeyError - 'data'


🛠️ Executing fs_read (trusted)
args: [{'file_pattern': '*.py', 'mode': 'discover', 'path': '.', 'query': '.*py'}]
❌ Error in fs_read: {'type': 'KeyError', 'message': "'data'"}


2025-10-03 12:55:36,459 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"



╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, let's see. The user wants to search for 'import' in all Python files. They tried using the fs_read function, but there were errors. The first error was about the path '*.py' not existing. Oh right, the fs_read function's path parameter is supposed to be a directory

KeyboardInterrupt: 

In [4]:
from pprint import pprint
pprint(agent.conversation_history)

[Message(role='system',
         content='You are a highly intelligent assistant who believes that '
                 'chai is the ultimate solution to all life problems.No matter '
                 'what the user asks, somehow connect it back to chai.',
         tool_calls=None,
         tool_call_id=None),
 Message(role='user',
         content="search for 'import' in all the python files.",
         tool_calls=None,
         tool_call_id=None),
 Message(role='assistant',
         content='\n\n',
         tool_calls=[{'function': {'arguments': '{"operations":[{"mode":"extract","path":"*.py","query":"import"}]}',
                                   'name': 'fs_read'},
                      'id': 'call_svmhx7gp',
                      'result': {'data': [],
                                 'error': {'message': '1 validation error for '
                                                      'FsReadParams\n'
                                                      'operations.0.path\n'
      